### Сверка по справкам (условным начислениям), импортированных в журнал проводок 1С.

##### Полуавтоматический контроль.

Описание:
1) pl - "плоский" (журнал проводок)

2) df1 - Бонус Малус

3) df3 - Справка резерв зарплаты водителей

Также по аналогии можно добавлять иные справки.

Логика работы следующая: Мы берем наименования справок, которые хотим проверить, фильтруем определенную колонку в Журнале проводок  по наименованию и записывает на отдельный лист. На другой лист записываем непосредственно справку. 
После сохранения файла  - идем в него и вручную проверяем корретность занесения статей затрат, сумм, кост-центров, наименования контргагентов.

In [17]:
import pandas as pd
from datetime import datetime
from pathlib import Path
import openpyxl

In [2]:
#Сначала читаем плоский
pl = pd.read_excel(r'S:\Actual\Controlling\Плоский для P&L_2022.xlsm', sheet_name='P&L')

### 01 Бонус-малус 2022 12.xlsx

In [3]:
#указываем путь к папке со справками
file_path = Path(r"S:\Actual\_Reporting 2022\22 12\Trs\УН") 

#в появившемся окне вводим название необходимой справки по которой будем фильтровать таблицу
file_name = input("Enter the name of the file: ")  
file = file_path / file_name

#Читаем справку, записываем в переменную
df1 = pd.read_excel(file)

# remove top 4 rows
df1 = df1.iloc[4:]

Enter the name of the file: 01 Бонус-малус 2022 12.xlsx


In [4]:
# 2) Filter another table by it

filtered_pl_1 = pl[pl['Комментарий'].notna() & pl['Комментарий'].str.contains(file_name)]

### 00 Справка резерв зарплаты водителей 2022 12

In [25]:
file_name_3 = input("Enter the name of the file: ")
file_3 = file_path / file_name_3

df3 = pd.read_excel(file_3)


Enter the name of the file: 00 Справка резерв зарплаты водителей 2022 12.xlsx


In [6]:
# 2) Filter another table by it

filtered_pl_2 = pl[pl['Комментарий'].notna() & pl['Комментарий'].str.contains(file_name_3)]

## 02 рекласс T-T0 202Х

In [43]:
file_name_4 = input("Enter the name of the file: ")
file_4 = file_path / file_name_4

df4 = pd.read_excel(file_4)


Enter the name of the file: 02 рекласс T-T0 2022 12_New.xlsx


In [44]:
# remove top 4 rows
df4.columns = df4.iloc[4]
df4 = df4.iloc[4:]

df4.head(3)

4,№ п/п,Контрагент / Заказчик,Номенклатура / Наименование,Сумма,Кост-центр,Счет,Аналитика,Статья затрат,Расчетная сумма\nНе выводиться на печать
4,№ п/п,Контрагент / Заказчик,Номенклатура / Наименование,Сумма,Кост-центр,Счет,Аналитика,Статья затрат,Расчетная сумма\nНе выводиться на печать
5,1,УСЛ_Корректировка,УСЛ_Корректировка,-2901.94,041_RCE1070_NOTABLE_BE_T Топливо,У20,Данон,ГСМ (Ad blue),-2901.94
6,2,УСЛ_Корректировка,УСЛ_Корректировка,2901.94,041_RCE1070_NOTABLE_BE_T0 Топливо,У20,Данон,ГСМ (Ad blue),2901.94


In [8]:
# 2) Filter another table by it

filtered_pl_4 = pl[pl['Комментарий'].notna() & pl['Комментарий'].str.contains(file_name_4)]

## 04 рекласс Subco 202Х

In [9]:
file_name_5 = input("Enter the name of the file: ")
file_5 = file_path / file_name_5

df5 = pd.read_excel(file_5)

# remove top 4 rows
df5 = df5.iloc[4:]

Enter the name of the file: 04 рекласс Subco 2022 12_V0.xlsx


In [10]:
# 2) Filter another table by it

filtered_pl_5 = pl[pl['Комментарий'].notna() & pl['Комментарий'].str.contains(file_name_5)]

## Объединим все датафреймы в один excel файл

In [14]:
#Объедините фреймы данных в один файл Excel на разных листах

with pd.ExcelWriter(r"S:\Actual\_Reporting 2022\22 12\Trs\УН\combined_file.xlsx") as writer:
    df1.to_excel(writer, sheet_name='01_БонусМ')
    filtered_pl_1.to_excel(writer, sheet_name='Плоский_БМ')
    df3.to_excel(writer, sheet_name='Спр_УН_зп_вод')
    filtered_pl_2.to_excel(writer, sheet_name='Плоский_ЗП_вод')
    df4.to_excel(writer, sheet_name='Спр_02_реклассТ_Т0')
    filtered_pl_4.to_excel(writer, sheet_name='Плоский_02_рекласс_T-T0')
    df5.to_excel(writer, sheet_name='Спр_04_рекласс_Subco')
    filtered_pl_5.to_excel(writer, sheet_name='Плоский_04 рекласс_Subco')
    
    
# Этот код использует класс pd.ExcelWriter для создания файла Excel 
# с указанным путем к файлу (r "S:\Actual\_Reporting 2022\22 12\Trs\УН\combined_file.xlsx") 
# и записи нескольких DataFrames на разные листы файла Excel 
    


#### Форматирование файла - добавление шрифтов и т.д.

In [22]:
# Загрузим файл Excel
workbook = openpyxl.load_workbook(r"S:\Actual\_Reporting 2022\22 12\Trs\УН\combined_file.xlsx")

# Определим величину шрифта
font = openpyxl.styles.fonts.Font(size=9)

# Пройдемся циклом по всем листам рабочей книги
for sheet in workbook:
    # Установим шрифт
    sheet.sheet_properties.pageSetUpPr.fitToPage = True
    sheet.sheet_properties.pageSetUpPr.fitToHeight = 1
    sheet.sheet_properties.pageSetUpPr.fitToWidth = 1
    for col in sheet.columns:
        for cell in col:
            cell.font = font
    # Сохраним книгу
    workbook.save(r"S:\Actual\_Reporting 2022\22 12\Trs\УН\combined_file.xlsx")
